# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [48]:
import pandas as pd
import plotly.offline as py
from plotly.subplots import make_subplots
from plotly import tools
import plotly.graph_objs as go
import warnings
from ipywidgets import interact
import numpy as np

warnings.filterwarnings('ignore')
py.init_notebook_mode(connected=True)

In [3]:
data = pd.read_excel('./Online Retail.xlsx')
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [4]:
data.shape

(396034, 9)

In [60]:
data.nunique()

InvoiceNo      18389
InvoiceDate    17155
StockCode       3658
Description     3640
Quantity         297
UnitPrice        355
Revenue         2693
CustomerID      4328
Country           35
dtype: int64

In [5]:
data.dtypes

InvoiceNo               int64
InvoiceDate    datetime64[ns]
StockCode              object
Description            object
Quantity                int64
UnitPrice             float64
Revenue               float64
CustomerID              int64
Country                object
dtype: object

#### 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [6]:
# April 2011 mask
import datetime as dt
mask_april_2011 = (data['InvoiceDate'].dt.month ==4) & (data['InvoiceDate'].dt.year ==2011)

In [7]:
# Country mask
mask_exclude_UK = ~data['Country'].str.contains('United Kingdom')

In [8]:
masks = [mask_april_2011, mask_exclude_UK]
grouping = 'Country'
cols = ['Quantity', 'Revenue']
subset1 = data.loc[np.logical_and.reduce(masks), cols + [grouping]].groupby(grouping).sum()

In [10]:
import plotly.express as px

fig = px.bar(data_frame = subset1, barmode='group',
            labels = {
                'value':'Total',
                'variable':'Measure'
            },
            title = 'Total Quantity and Revenue by Country in April 2011',
            width=800,
            height=600)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified", yaxis=dict(tickformat=",.0f"), title_x=0.5)
fig.show()

#### 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [13]:
mask_jan_to_may_2011 = (data['InvoiceDate'].dt.month >=1) & (data['InvoiceDate'].dt.month <=5) & (data['InvoiceDate'].dt.year ==2011)
mask_france =  data['Country'] == 'France'

In [20]:
masks = [mask_jan_to_may_2011, mask_france]
cols = ['Quantity', 'Revenue', 'InvoiceDate']
subset2 = data.loc[np.logical_and.reduce(masks), cols].groupby([data['InvoiceDate'].dt.date]).sum()

In [27]:
fig = px.line(data_frame = subset2,
            labels = {
                'value':'Total',
                'variable':'Measure',
                'InvoiceDate': 'Date'
            },
            title = 'Total Quantity and Revenue in France: Jan-May 2011',
            width=800,
            height=600)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified", yaxis=dict(tickformat=",.0f"), title_x=0.5)
fig.show()

#### 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [29]:
mask_partybunting = data['Description'].str.contains('PARTY BUNTING')

In [30]:
masks = [mask_partybunting]
grouping = 'Country'
cols = ['Quantity', 'UnitPrice']
subset3 = data.loc[np.logical_and.reduce(masks), cols + [grouping]].groupby(grouping).mean()

In [43]:
fig = px.scatter(data_frame = subset3, x='Quantity', y='UnitPrice', color = subset3.index,
            labels = {
                'UnitPrice': 'Average Unit Price',
                'Quantity': 'Average Quantity'
            },
            title = 'Average Quantity vs. Average Unit price for Party Bunting',
            width=800,
            height=600)
fig.update_layout(yaxis=dict(tickformat=",.1f"), title_x=0.5)
fig.show()

#### 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [82]:
select_countries = ['EIRE','France', 'Germany', 'Netherlands']
mask_select_countries = data['Country'].isin(select_countries)

In [83]:
masks = [mask_select_countries]
grouping = ['Country', 'InvoiceNo']
cols = ['Quantity']
subset4 = data.loc[np.logical_and.reduce(masks), cols + grouping].groupby(grouping).sum()
subset4.reset_index(level=1, inplace=True)

In [90]:
xbins_size = {
    'France': 50,
    'Germany': 50,
    'EIRE': 100,
    'Netherlands': 500
}

fig = make_subplots(rows=2, cols=2, subplot_titles = select_countries)

for i, country in enumerate(select_countries):
    fig.add_trace(
        go.Histogram(x = subset4.loc[subset4.index == country].Quantity, xbins=dict(size = xbins_size[country]), name=country),
        row=i//2+1, col=i%2+1
    )

fig.update_layout(height=600, width=800, title_text="Distribution of Quantity per Invoice for given Countries")
fig.update_xaxes(title = 'Quantity')
fig.update_yaxes(title = '# of invoices')

fig.show()

#### 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [92]:
# Variables
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [97]:
# Transform the data
rvn_country = data[data['Country'].isin(country_list) &
                   data['Description'].isin(product_list)]
rvn_country

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
179,539320,2010-12-16 19:16:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.80,14911,EIRE
198,539722,2010-12-21 13:45:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.80,14911,EIRE
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.80,14646,Netherlands
322,541979,2011-01-24 14:54:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE
367,542777,2011-02-01 08:31:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE
...,...,...,...,...,...,...,...,...,...
143798,580753,2011-12-06 10:00:00,22423,REGENCY CAKESTAND 3 TIER,1,12.75,12.75,12682,France
143808,581098,2011-12-07 11:42:00,22423,REGENCY CAKESTAND 3 TIER,16,10.95,175.20,14911,EIRE
143809,581175,2011-12-07 15:16:00,22423,REGENCY CAKESTAND 3 TIER,32,10.95,350.40,14646,Netherlands
143810,581179,2011-12-07 15:43:00,22423,REGENCY CAKESTAND 3 TIER,64,10.95,700.80,12471,Germany


In [98]:
rvn_country = rvn_country.pivot_table(index='Country', columns='Description', values='Revenue', aggfunc='sum')
rvn_country

Description,CREAM HANGING HEART T-LIGHT HOLDER,JUMBO BAG RED RETROSPOT,REGENCY CAKESTAND 3 TIER
Country,,,
EIRE,2740.80,278.72,7388.55
France,131.75,903.37,2816.85
Germany,35.40,1072.76,9061.95
Netherlands,1167.00,3468.00,3166.35


In [105]:
fig = px.bar(data_frame = rvn_country,
            labels = {
                'value':'Total Revenue'
            },
            title = 'Total Revenue per Country for given Products',
            width=800,
            height=600,
            barmode='group')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="closest", yaxis=dict(tickformat=",.0f"), title_x=0.5)
fig.show()

#### 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [106]:
# Transform the data
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [107]:
# Variables
years = uk['Year'].unique()
months = uk['Month'].unique()
months.sort()

In [135]:
# Interactive menu
@interact(year=years, 
          month=months)

# Update function
def linechart(year=2011, month=4):
    fig = px.line(data_frame = uk[(uk.Year == year) & (uk.Month == month)].groupby('Day').sum(), y='Quantity',
                labels = {
                    
                },
                title = f'Daily quantity sold in the UK in {dt.datetime.strptime(str(month), "%m").strftime("%B")} {year}',
                width=800,
                height=600)
    fig.update_layout(yaxis=dict(tickformat=",.0f"), title_x=0.5)
    fig.show()

interactive(children=(Dropdown(description='year', index=1, options=(2010, 2011), value=2011), Dropdown(descri…

#### 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [136]:
# Transform the data
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = data.groupby('Description').agg(agg_func)

In [138]:
products.head()

,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
Description,,,,,
4 PURPLE FLOCK DINNER CANDLES,36,140,2.324359,270.76,31
50'S CHRISTMAS GIFT BAG LARGE,111,1909,1.248108,2302.25,108
DOLLY GIRL BEAKER,133,2398,1.243841,2759.50,100
I LOVE LONDON MINI BACKPACK,69,361,4.138732,1458.15,58
NINE DRAWER OFFICE TIDY,30,55,14.791935,792.85,29


In [159]:
# Interactive menu
@interact(invoices=(products['InvoiceNo'].min(), products['InvoiceNo'].max(), 10), 
          customers=(products['CustomerID'].min(), products['CustomerID'].max(), 10))

# Update function
def scatter(invoices=1, customers=1):
    fig = px.scatter(data_frame = products, x='InvoiceNo', y='CustomerID', hover_name=products.index,
                range_x=[invoices,products['InvoiceNo'].max()],
                range_y=[customers, products['CustomerID'].max()],
                labels = {
                    'CustomerID':'# of Customers',
                    'InvoiceNo':'# of Invoices'
                },
                title = 'Number of Invoices vs. Number of Customers for each Product',
                width=800,
                height=600)
    fig.update_layout(yaxis=dict(tickformat=",.0f"), title_x=0.5)
    fig.show()

    px.scatter()

interactive(children=(IntSlider(value=1, description='invoices', max=1978, min=1, step=10), IntSlider(value=1,…

In [158]:
import ipywidgets as widgets
from IPython.display import display

# Interactive menu

invoices = widgets.IntSlider(min = products['InvoiceNo'].min(), max = products['InvoiceNo'].max(), step =10, description = '# of Invoices')
customers = widgets.IntSlider(min = products['CustomerID'].min(), max = products['CustomerID'].max(), step =10, description = '# of Customers')
ui = widgets.HBox([invoices, customers])

# Update function
def scatter(invoices=1, customers=1):
    fig = px.scatter(data_frame = products, x='InvoiceNo', y='CustomerID', hover_name=products.index,
                range_x=[invoices,products['InvoiceNo'].max()],
                range_y=[customers, products['CustomerID'].max()],
                labels = {
                    'CustomerID':'# of Customers',
                    'InvoiceNo':'# of Invoices'
                },
                title = 'Number of Invoices vs. Number of Customers for each Product',
                width=800,
                height=600)
    fig.update_layout(yaxis=dict(tickformat=",.0f"), title_x=0.5)
    fig.show()

    px.scatter()

out = widgets.interactive_output(scatter, {'invoices': invoices, 'customers': customers})

display(ui, out)

Output()

#### 8. Create an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [168]:
product = widgets.Text(
    value='JUMBO',
    placeholder='Type anything',
    description='Product:',
    disabled=False
)

# Update function
def chart(product='JUMBO'):
    fig = px.bar(data_frame = data.loc[data.Description.str.contains(product), ['Description', 'Revenue']].groupby('Description').sum(),
                #x='Description',
                #y='Revenue',
                labels = {
                    'variable':'',
                    'value': 'Total Revenue',
                    'Description': 'Product'
                },
                title = f'Total Revenue by Product for "{product}" Products',
                width=800,
                height=600)
    fig.update_traces(hovertemplate=None)
    fig.update_layout(hovermode="closest", yaxis=dict(tickformat=",.0f"), title_x=0.5)
    fig.show()

    px.scatter()

out = widgets.interactive_output(chart, {'product': product})

display(product, out)

Text(value='JUMBO', description='Product:', placeholder='Type anything')

Output()